In [1]:
#@title Select Environment { display-mode: "form" }
env_id = "3DBall" #@param ['Basic', '3DBall', '3DBallHard', 'GridWorld', 'Hallway', 'VisualHallway', 'CrawlerDynamicTarget', 'CrawlerStaticTarget', 'Bouncer', 'SoccerTwos', 'PushBlock', 'VisualPushBlock', 'WallJump', 'Tennis', 'Reacher', 'Pyramids', 'VisualPyramids', 'Walker', 'FoodCollector', 'VisualFoodCollector', 'StrikersVsGoalie', 'WormStaticTarget', 'WormDynamicTarget']

In [2]:
# -----------------
# This code is used to close an env that might not have been closed before
try:
  env.close()
except:
  pass
# -----------------

from mlagents_envs.registry import default_registry
from mlagents_envs.pettingzoo_wrapper import UnityToPettingZooWrapper

unity_env = default_registry[env_id].make()
env = UnityToPettingZooWrapper(unity_env)
print("Loaded environment")

Loaded environment


In [3]:
num_cycles = 10

env.reset()
for agent in env.agent_iter(env.num_agents * num_cycles):
    prev_observe, reward, done, info = env.last()
    if done:
        action = None
    elif isinstance(prev_observe, dict) and 'action_mask' in prev_observe:
        action = random.choice(np.flatnonzero(prev_observe['action_mask']))
    else:
        action = env.action_spaces[agent].sample()
    env.step(action)
print("Done stepping")

done stepping


In [4]:
env.close()
print("Closed environment")

Closed environment
